In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
from operator import add
import builtins
from google.cloud import storage
from collections import defaultdict

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')



In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
!pip install nltk

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
def open_gcp(file_name,dir_name):
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob(f'postings_gcp{dir_name}/' + file_name)
    return blob.open('rb')

def read_pickle(file_name,dir_name):
    stream = open_gcp(file_name+".pickle",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

def read_pkl(file_name,dir_name):
    stream = open_gcp(file_name+".pkl",dir_name)
    pick = pickle.load(stream)
    stream.close()
    return pick

In [ ]:
def write_pickle(object_to_write,object_name ,bucket_name,dir_name):
    with open(f"{object_name}.pickle", "wb") as f:
        pickle.dump(object_to_write, f)
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp{dir_name}/{object_name}.pickle")
    blob_posting_locs.upload_from_filename(f"{object_name}.pickle")

def write_pkl(object_to_write,object_name ,bucket_name,dir_name):
    with open(f"{object_name}.pkl", "wb") as f:
        pickle.dump(object_to_write, f)
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp{dir_name}/{object_name}.pkl")
    blob_posting_locs.upload_from_filename(f"{object_name}.pkl")



In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
bucket_name = '318457645' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)

In [ ]:
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import *

In [ ]:
full_path = "gs://wikidata_preprocessed/*"


In [ ]:
df_data = spark.read.parquet(full_path)
id_text = df_data.select('id','text')
id_text = id_text.rdd
inverted_text = InvertedIndex()


In [ ]:
porter = PorterStemmer()

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(id, text):

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [x for x in tokens if x not in all_stopwords]
    tokens = [porter.stem(x) for x in tokens]

    tuples = []
    tf_dict = Counter(tokens)
    res = []
    [res.append(x) for x in tokens if x not in res]
    for t in res:
        tuples.append((t,(id,tf_dict[t])))
    return tuples

In [ ]:
def count_doc_len(doc_id, text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [x for x in tokens if x not in all_stopwords]
    tokens = [porter.stem(x) for x in tokens]
    return (doc_id,len(tokens))

In [ ]:
def tf_for_term_id(doc_id, text):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = [x for x in tokens if x not in all_stopwords]
    tokens = [porter.stem(x) for x in tokens]
    tuples = []
    tf_dict = Counter(tokens)
    res = []
    [res.append(x) for x in tokens if x not in res]
    for t in res:
        tuples.append((t,tf_dict[t]))
    return tuples

In [ ]:
word_counts_text = id_text.flatMap(lambda x: word_count(x[0], x[1]))


In [ ]:
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

In [ ]:
postings_text = word_counts_text.groupByKey().mapValues(reduce_word_counts)
postings_filtered_text = postings_text.filter(lambda x: len(x[1])>50)


In [ ]:
total_terms_text = postings_filtered_text.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_text.term_total = total_terms_text.collectAsMap()

In [ ]:
def calculate_df(postings):
    rdd2 = postings.map(lambda x: (x[0], len(x[1]))) 
    return rdd2

In [ ]:
w2df_text = calculate_df(postings_filtered_text)

w2df_text_dict = w2df_text.collectAsMap()

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings,dir_name):
    rdd2 = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    rdd3 = rdd2.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name,dir_name))
    return rdd3

In [ ]:
postings_locs_text = partition_postings_and_write(postings_filtered_text,'text').collect()


In [ ]:
super_posting_locs_text = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcptext'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_text[k].extend(v)

In [ ]:
inverted_text.posting_locs = super_posting_locs_text
inverted_text.df = w2df_text_dict

In [ ]:
def calculate_tfidf_doc(tf_term_doc, index):
    DL = index.DL
    N = len(DL)
    df_dict = index.df
    doc_term_tfidf = tf_term_doc.map(lambda x : (x[0] , math.sqrt(builtins.sum([math.pow((tf/DL[x[0]]) * (math.log(N/df_dict[term],10)),2) for term, tf in x[1] if term in df_dict and x[0] in index.DL]))))
    return doc_term_tfidf

In [ ]:
tf_term_doc_text = id_text.map(lambda x : (x[0], tf_for_term_id(x[0], x[1])))
tfidf_text = calculate_tfidf_doc(tf_term_doc_text, inverted_text)
tfidf_dict_text = tfidf_text.collectAsMap()

In [ ]:
write_pickle(inverted_text.DL, "text_DL",bucket_name,"text" )

In [ ]:
write_pickle(inverted_text.term_total, "text_term_total",bucket_name,"text")

In [ ]:
write_pickle(inverted_text.vec_len_doc, "vec_len_total",bucket_name,"text")

In [ ]:
inverted_text.write_index('.', 'index_text')
index_src = "index_text.pkl"
index_dst = f'gs://{bucket_name}/postings_gcptext/{index_src}'
!gsutil cp $index_src $index_dst